In [2]:
import pandas as pd

# ---------------------------------------------------------
# Helper: read gzipped TSV with minimal boilerplate
# ---------------------------------------------------------
def read_tsv_gz(path, names, dtypes):
    return pd.read_csv(
        path,
        sep="\t",
        header=None,
        names=names,
        dtype=dtypes,
        compression="gzip",
        memory_map=True,   # a little faster on big files
    )

# ---------------------------------------------------------
# 1. pages.txt.gz  →  page_id · page_namespace · page_title · page_is_redirect
# ---------------------------------------------------------
pages_cols = ["page_id", "page_namespace", "page_title", "page_is_redirect"]
pages_dtypes = {
    "page_id": "int32",
    "page_namespace": "int16",
    "page_title": "string",
    "page_is_redirect": "int8",
}
pages = read_tsv_gz("pages.txt.gz", pages_cols, pages_dtypes)
pages.insert(0, "row", range(len(pages)))
display(pages.head())

# ---------------------------------------------------------
# 2. links.txt.gz  →  pl_from · pl_from_namespace · pl_target_id
# ---------------------------------------------------------
links_cols = ["pl_from", "pl_from_namespace", "pl_target_id"]
links_dtypes = {
    "pl_from": "int32",
    "pl_from_namespace": "int16",
    "pl_target_id": "int32",
}
links = read_tsv_gz("links.txt.gz", links_cols, links_dtypes)
links.insert(0, "row", range(len(links)))
display(links.head())

# ---------------------------------------------------------
# 3. redirects.txt.gz  →  rd_from · rd_namespace · rd_title
# ---------------------------------------------------------
redir_cols = ["rd_from", "rd_namespace", "rd_title"]
redir_dtypes = {
    "rd_from": "int32",
    "rd_namespace": "int16",
    "rd_title": "string",
}
redirects = read_tsv_gz("redirects.txt.gz", redir_cols, redir_dtypes)
redirects.insert(0, "row", range(len(redirects)))
display(redirects.head())

# ---------------------------------------------------------
# 4. linktarget.txt.gz  →  lt_id · lt_namespace · lt_title
# ---------------------------------------------------------
lt_cols = ["lt_id", "lt_namespace", "lt_title"]
lt_dtypes = {
    "lt_id": "int32",
    "lt_namespace": "int16",
    "lt_title": "string",
}
linktargets = read_tsv_gz("linktarget.txt.gz", lt_cols, lt_dtypes)
linktargets.insert(0, "row", range(len(linktargets)))
print("pages rows:", len(pages))
print("links rows:", len(links))
print("redirects rows:", len(redirects))
print("linktargets rows:", len(linktargets))
display(linktargets.head())

,row,page_id,page_namespace,page_title,page_is_redirect
0,0,1,0,April,0
1,1,2,0,August,0
2,2,3,2,Angela,0
3,3,4,3,Angela,0
4,4,5,2,Anthere,0


,row,pl_from,pl_from_namespace,pl_target_id
0,0,128281,0,4
1,1,866814,0,30
2,2,869126,0,30
3,3,89,0,59
4,4,3196,0,59


,row,rd_from,rd_namespace,rd_title
0,0,24,0,Catharism
1,1,41,0,United_States_customary_units
2,2,46,0,AS
3,3,61,0,Animal
4,4,79,4,Proposed_very_good_articles


pages rows: 884909
links rows: 24174037
redirects rows: 124256
linktargets rows: 2554643


,row,lt_id,lt_namespace,lt_title
0,0,2045505,0,!
1,1,2255546,0,!!
2,2,70680,0,!!!
3,3,773926,0,!!!Fuck_You!!!_and_Then_Some
4,4,2038772,0,!!!_(disambiguation)


In [10]:
import duckdb

# --- call this once to share the same connection everywhere -------------
con = duckdb.connect()
con.register("pages", pages)          # make sure `pages` DataFrame is registered

def page_id_from_title(ns: int, title: str) -> int | None:

    res = con.execute(
        """
        SELECT page_id
        FROM pages
        WHERE page_namespace = $1
          AND page_title     = $2
        LIMIT 1;
        """,
        [ns, title]
    ).fetchone()
    return res[0] if res else None


# ---------------- example usage ----------------
pid = page_id_from_title(0, "Table_(furniture)")
print("page_id =", pid)

page_id = 243370


In [4]:
import duckdb, pandas as pd

SOURCE_ID = 297681   # ← choose the page you want to inspect

con = duckdb.connect()
con.register("pages",       pages)
con.register("links",       links)
con.register("linktargets", linktargets)

query = """
SELECT
    pl.pl_from          AS src_id,       -- from links
    pl.pl_target_id     AS raw_tgt_id,   -- from links
    tgt.page_id         AS tgt_id,       -- resolved page ID
    lt.lt_title         AS tgt_title,    -- title from linktargets
    tgt.page_namespace  AS tgt_ns        -- namespace of resolved page
FROM links        AS pl
JOIN linktargets  AS lt  ON lt.lt_id            = pl.pl_target_id
JOIN pages        AS tgt ON tgt.page_namespace  = lt.lt_namespace
                        AND tgt.page_title      = lt.lt_title
WHERE pl.pl_from = $1
ORDER BY src_id, tgt_ns, tgt_title;
"""

df = con.execute(query, [SOURCE_ID]).df()
print(f"Total links found: {len(df):,}")
pd.set_option("display.max_columns", None)
display(df.head(742))      # show the first 20 outgoing links

Total links found: 7


,src_id,raw_tgt_id,tgt_id,tgt_title,tgt_ns
0,297681,219770,27486,Cloth,0
1,297681,271558,4061,Face,0
2,297681,281891,13833,Mouth,0
3,297681,13061,4892,Rectangle,0
4,297681,269189,243370,Table_(furniture),0
5,297681,416997,25778,Towel,0
6,297681,206177,8705,Stub,4


In [5]:
import duckdb, pandas as pd

con = duckdb.connect()
con.register("pages",       pages)        # page_id, page_namespace, page_title, …
con.register("links",       links)        # pl_from, pl_from_namespace, pl_target_id
con.register("linktargets", linktargets)  # lt_id, lt_namespace, lt_title

query = """
SELECT
    pl.pl_from          AS src_id,       -- from links
    pl.pl_target_id     AS raw_tgt_id,   -- from links
    tgt.page_id         AS tgt_id,       -- resolved page ID
    lt.lt_title         AS tgt_title,    -- title from linktargets
    tgt.page_namespace  AS tgt_ns        -- namespace of resolved page
FROM links        AS pl
JOIN linktargets  AS lt  ON lt.lt_id            = pl.pl_target_id
JOIN pages        AS tgt ON tgt.page_namespace  = lt.lt_namespace
                        AND tgt.page_title      = lt.lt_title
ORDER BY src_id, tgt_ns, tgt_title;
"""

link_edges_df = con.execute(query).df()

print(f"Total link rows: {len(link_edges_df):,}")
pd.set_option("display.max_columns", None)
display(link_edges_df.head(100))

Total link rows: 18,193,806


,src_id,raw_tgt_id,tgt_id,tgt_title,tgt_ns
0,1,365979,39168,1320,0
1,1,365980,8495,1502,0
2,1,365981,11676,1509,0
3,1,324652,10141,1519,0
4,1,365982,8497,1533,0
...,...,...,...,...,...
95,1,345605,9878,April_25,0
96,1,334012,9879,April_26,0
97,1,224770,9881,April_27,0
98,1,224771,5148,April_28,0


In [6]:
# Count missing (NaN / NULL) values in every column
null_counts = link_edges_df.isna().sum()

print("Null-value count per column:")
for col, n in null_counts.items():
    print(f"  {col:<10}: {n:,}")

Null-value count per column:
  src_id    : 0
  raw_tgt_id: 0
  tgt_id    : 0
  tgt_title : 0
  tgt_ns    : 0


In [7]:
# get the two columns, drop duplicates
unique_pairs = link_edges_df[["src_id", "tgt_id"]].drop_duplicates()

print(f"Unique (src_id → tgt_id) pairs: {len(unique_pairs):,}")
display(unique_pairs.head(20))      # show first 20 pairs

Unique (src_id → tgt_id) pairs: 18,193,806


,src_id,tgt_id
0,1,39168
1,1,8495
2,1,11676
3,1,10141
4,1,8497
5,1,21063
6,1,39092
7,1,5155
8,1,19968
9,1,16325


In [8]:
merged_cols = [
    "src_id",
    "tgt_id"
]
merged_dtypes = {
    "src_id":   "int32",
    "tgt_id":   "int32",
}

edges_df = read_tsv_gz("edges.ids_only.txt.gz",
                        merged_cols, merged_dtypes)
edges_df.insert(0, "row", range(len(edges_df)))

print("total:", len(edges_df))
display(edges_df.head())

total: 18193825


,row,src_id,tgt_id
0,0,128281,4836
1,1,866814,90050
2,2,869126,90050
3,3,89,64585
4,4,3196,64585


In [9]:
import duckdb
import pandas as pd

def edges_for_source_duck(wanted_id: int,
                          edges_df: pd.DataFrame,
                          pages_df: pd.DataFrame) -> pd.DataFrame:
    """
    Return an edge list limited to one source page, with human-readable columns.

    Columns in result:
      src_id · src_ns · src_title · tgt_id · tgt_ns · tgt_title
    """
    con = duckdb.connect()
    con.register("edges",  edges_df)
    con.register("pages",  pages_df)

    query = """
    SELECT
        src.page_id        AS src_id,
        src.page_namespace AS src_ns,
        src.page_title     AS src_title,
        tgt.page_id        AS tgt_id,
        tgt.page_namespace AS tgt_ns,
        tgt.page_title     AS tgt_title
    FROM edges   AS e
    JOIN pages   AS src ON src.page_id = e.src_id
    JOIN pages   AS tgt ON tgt.page_id = e.tgt_id
    WHERE src.page_id = ?
    ORDER BY tgt_ns, tgt_title;
    """

    return con.execute(query, [wanted_id]).df()

# ────────── example usage ──────────
result_df = edges_for_source_duck(297681, edges_df, pages)
print(f"{len(result_df):,} outgoing links")
display(result_df.head(10))

7 outgoing links


,src_id,src_ns,src_title,tgt_id,tgt_ns,tgt_title
0,297681,0,Napkin,27486,0,Cloth
1,297681,0,Napkin,4061,0,Face
2,297681,0,Napkin,13833,0,Mouth
3,297681,0,Napkin,4892,0,Rectangle
4,297681,0,Napkin,243370,0,Table_(furniture)
5,297681,0,Napkin,25778,0,Towel
6,297681,0,Napkin,8705,4,Stub
